In [17]:
import pandas as pd
import numpy as np
import warnings
from functions import metrics
import csv
from sklearn.exceptions import ConvergenceWarning
from functions.formatting import get_subgroup_str
from itertools import product, combinations
import matplotlib.pyplot as plt
import ast

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [18]:
omit_demographics = True 
use_gerryfair = False 

In [19]:
df = pd.read_csv('./data/preprocessed.csv')
df.drop(['umich_user_id', 'Unnamed: 0'], axis=1, inplace=True)

In [20]:
X = df.drop('completed', axis=1)
y = df['completed']

In [21]:
X['black_female'] = (X['black'] == 1) & (X['female'] == 1)

#### Calculate false positive rates for the given subgroup

This function takes in the subgroup to test for as a list of tuples. We use logistic regression and 10-fold cross validation to calculate the average FPR and AUC for the given subgroup. 

Returns a tuple containing FPR average, FPR standard deviation, AUC average, AUC standard deviation, and number of students in a given subgroup

#### Compute all subgroups

This function generates all possible subgroups given the list of protected columns

In [22]:
demo_groups = [['US', 'intl'], ['bachelor_obtained', 'no_bachelor_obtained', 'education_na'], ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na'], ['male', 'female', 'gender_na', 'gender_other']]

In [23]:
def compute_combos():
    ret = []
    for i in range(1,len(demo_groups)+1):
        g = list(combinations(demo_groups, i))
        for demos in g:
            ret += list(product(*demos))

    return ret

In [24]:
combos = compute_combos()

In [25]:
combos.sort(key=len)
# combos
combos.insert(0, ())

In [ ]:
combos

In [26]:
demographics = ['US', 'intl', 'bachelor_obtained', 'no_bachelor_obtained', 'education_na', 'white', 'black', 'asian', 'latinx', 'race_others', 'race_na', 'male', 'female', 'gender_na', 'gender_other']
# protected=['US', 'intl', 'bachelor_obtained', 'no_bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female']
# protected = demographics
# protected = ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na', 'male', 'female', 'gender_na', 'gender_other']
# protected = ['black', 'female']
protected = ['black_female']

In [ ]:
# protected = ['US', 'intl', 'white', 'black', 'asian', 'latinx', 'race_others', 'race_na']

In [ ]:
res = metrics.calc_metrics(X, y, combos, omit_demographics=True, demographics=demographics, protected=protected, is_gerryfair=False, iters=300)

In [27]:
res = metrics.calc_metrics(X, y, combos, omit_demographics=True, demographics=demographics, protected=protected, is_gerryfair=True, iters=300)

iteration: 1
most accurate classifier accuracy: 0.20331491712707184, most acc-class unfairness: 0.008878212827288014, most acc-class size 0.026519337016574586
barrier
iteration: 2
barrier
iteration: 3
barrier
iteration: 4
barrier
iteration: 5
barrier
iteration: 6
barrier
iteration: 7
barrier
iteration: 8
barrier
iteration: 9
barrier
iteration: 10
barrier
iteration: 11
barrier
iteration: 12
barrier
iteration: 13
barrier
iteration: 14
barrier
iteration: 15
barrier
iteration: 16
barrier
iteration: 17
barrier
iteration: 18
barrier
iteration: 19
barrier
iteration: 20
barrier
iteration: 21
barrier
iteration: 22
barrier
iteration: 23
barrier
iteration: 24
barrier
iteration: 25
barrier
iteration: 26
barrier
iteration: 27
barrier
iteration: 28
barrier
iteration: 29
barrier
iteration: 30
barrier
iteration: 31
barrier
iteration: 32
barrier
iteration: 33
barrier
iteration: 34
barrier
iteration: 35
barrier
iteration: 36
barrier
iteration: 37
barrier
iteration: 38
barrier
iteration: 39
barrier
itera

/Users/viveksaravanan/Documents/research/algofair/algofair/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/viveksaravanan/Documents/research/algofair/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/viveksaravanan/Documents/research/algofair/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/viveksaravanan/Documents/research/algofair/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/viveksaravanan/Documents/research/algofair/algofair/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:198: R

In [ ]:
res = metrics.gf_learning_curve(X, y, combos, omit_demographics=True, demographics=demographics, protected=protected, is_gerryfair=True, iters=10)

In [ ]:
# res

Generate a list of all subgroup data and write it to a csv

In [28]:
# csv_file = f"./data/MTC508_subgroup_data_without_demographics.csv"
# csv_file = "test.csv"
csv_file = "blackfemale_proc.csv"

# Write the data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    fieldnames = [
        'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()  # Write the header row
    for row in res:
        writer.writerow(row)

In [ ]:
# data = {}

# data['overall'] = len(X)

# for c in combos[1:]:
#     masks = [X[name] == value for name, value in c]
#     final_mask = pd.concat(masks, axis=1).all(axis=1)
#     filter_X = X[final_mask]
#     data[get_subgroup_str(c)] = len(filter_X)




In [ ]:
# df = pd.DataFrame(list(data.items()), columns=['Key', 'Count'])

# # Specify the CSV file path
# csv_file_path = 'data/MTC508_data_counts.csv'

# # Write the DataFrame to a CSV file
# df.to_csv(csv_file_path, index=False)

In [12]:
subgroup_data = pd.read_csv('./data/MTC508_subgroup_data_without_demographics.csv')
fpr_data = {}
# loop through each subgroup
# for each subgroup, get the corresponding data
for index, row in subgroup_data.iterrows():
    # print(f"Index: {index}, fpr: {row['fpr_avg']}")
    fpr_data[row['subgroup']] = row['fpr_avg']



In [13]:
subgroup_data = pd.read_csv('./data/MTC508_subgroup_data_without_demographics.csv')
auc_data = {}
# loop through each subgroup
# for each subgroup, get the corresponding data
for index, row in subgroup_data.iterrows():
    # print(f"Index: {index}, fpr: {row['fpr_avg']}")
    auc_data[row['subgroup']] = row['auc_avg']

In [ ]:
print(auc_data)

In [ ]:
def get_data_iters(start, stop, step, attrs, protected):
    ret_val_auc = []
    ret_val_fpr = []
    data = []
    for i in range(start+1,stop+1,step):
        data.append(metrics.calc_metrics(X, y, combos, omit_demographics=True, demographics=demographics, protected=protected, is_gerryfair=True, iters=i))
    
    for a in attrs:
        aucs = []
        fprs = []
        for res in data:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    aucs.append(float(subgroup['auc_avg']))
                    fprs.append(float(subgroup['fpr_avg']))
                    break

        iterations = np.array(list(range(start, stop, step)))

        other_algorithm_auc = np.linspace(auc_data[a], auc_data[a], iterations.size)
        other_algorithm_fpr = np.linspace(fpr_data[a], fpr_data[a], iterations.size)

        ret_val_auc.append((iterations, aucs, other_algorithm_auc))
        ret_val_fpr.append((iterations, fprs, other_algorithm_fpr))
    
    return ret_val_auc, ret_val_fpr
        


In [ ]:
attrs = ['Overall', 'black', 'white', 'female', 'male', 'black, female']

In [ ]:
val_auc, val_fpr = get_data_iters(0, 301, 20, attrs, protected)
# val_auc, val_fpr = get_data_iters(0, 11, 5, attrs, protected)

In [ ]:
val_auc, val_fpr

In [ ]:
def save_results(data, name):
    arr = np.array(data)
    flat_data = arr.reshape(arr.shape[0], -1)

    # Create a DataFrame with columns representing each element of the 2D array
    df = pd.DataFrame(flat_data, columns=[f'Value_{i}' for i in range(flat_data.shape[1])])

    # Save DataFrame to CSV file
    df.to_csv('results/' + name, index=False)


In [ ]:
def read_results(name):
    read_df = pd.read_csv('results/' + name)

    # Convert the 'Values' column back to a 3D array
    read_array_data = read_df.values.reshape(read_df.shape[0], 3, -1)
    return read_array_data

In [ ]:
save_results(val_auc, 'race_gender_protected_auc.csv')

In [ ]:
data = read_results('race_gender_protected_auc.csv')

In [ ]:
import matplotlib.pyplot as plt

def build_combined_graph(val, attributes):
    # Set distinct colors for each attribute
    colors = ['b', 'g', 'r', 'c', 'm', 'y']  # You can add more colors if needed

    # Create a single plot
    plt.figure(figsize=(12, 6))  # Adjust the figure size as needed

    for i, attr in enumerate(attributes):
        x_axis, y_axis, other_algorithm_fpr = val[i]
        plt.plot(x_axis, y_axis, linestyle='-', label=f'GerryFair - {attr.capitalize()}', color=colors[i])
        plt.plot(x_axis, other_algorithm_fpr, linestyle='-', label=f'LR Without Gerryfair - {attr.capitalize()}', color=colors[i], alpha=0.5)

    # Set the y-axis range to 0 to 1
    plt.ylim(0, 1)

    plt.xlabel('Iterations')
    plt.ylabel('AUC')
    plt.title('Comparison of Algorithms - AUC vs Iterations (Race + Gender Protected)')

    # Move the legend outside the axes
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # Automatically adjust subplot parameters to give specified padding
    plt.tight_layout()

    plt.show()

attributes = ["Overall", "Black", "White", "Female", "Male", "Black + Female"]

build_combined_graph(data, attributes)


In [ ]:
def build_graph_gamma(iters, gammas, attrs):
    results = []
    for g in gammas:
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=iters+1, gamma=g))
    
    data = results
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['auc_avg']))
                    break
        
        fprs = np.array(fprs)
        gammas = np.array(gammas)

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], gammas.size) 

        plt.plot(gammas, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(gammas, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Gamma')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Gamma For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_gamma(5, [.002, .005, .01, .02, .05, .1], ['Overall', 'black', 'female', 'black, female'])

In [ ]:
len(data)

In [ ]:
def gen_data_attrs(iters, attrs, sets):
    results = []
    for i in range(len(sets)):
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=demographics[:i+1], is_gerryfair=True, iters=iters+1, gamma=g))
    
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        num_attrs = np.array(list(range(1,len(demographics) + 1)))

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], len(demographics)) 

        plt.plot(num_attrs, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(num_attrs, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Number of Attributes')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Number of Attributes For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
gerryfair_groups = {
    'all': protected,
    'location': demo_groups[0], 
    'education': demo_groups[1][:-1], 
    'race': demo_groups[2][:-2], 
    'gender': demo_groups[3][:-2],
    'race+gender': demo_groups[2][:-2] + demo_groups[3][:-2]
    }

In [ ]:
def gen_data_categories(iters, attrs):
    for g in attrs:
        res = metrics.calc_metrics(X, y, combos, omit_demographics=False, demographics=demographics, protected=attrs[g], is_gerryfair=True, iters=iters)
        csv_file = f"./data/graph_attrs_{str(iters)}_{g}.csv"

        # Write the data to a CSV file
        with open(csv_file, 'w', newline='') as csvfile:
            fieldnames = [
                'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()  # Write the header row
            for row in res:
                writer.writerow(row)

In [ ]:
gen_data_categories(200, gerryfair_groups)

In [ ]:
demographics

In [ ]:
demo_groups[2][-2]

In [ ]:
protected

In [ ]:
part = df[(df['black'] == 1) & (df['female'] == 1)]
part['completed'].sum() / len(part['completed'])

In [ ]:
df.columns

In [ ]:
df = pd.read_csv('./data/MTC508_subgroup_data_without_demographics.csv')

In [ ]:
df[df['subgroup'] == 'Overall']['fpr_avg'][0]  - df[df['subgroup'] == 'no_bachelor_obtained, female']['fpr_avg']

In [ ]:
df[df['subgroup'] == 'white']['fpr_avg']